In [19]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/contextualize_best_hits")

library(data.table)

myo_meta <- read.csv("../../../data/myotis_meta.csv")
ortho_gene_mapping <- fread("../../../data/ortholog_mapping.csv", data.table = FALSE)
scaf_col <- grep("^yum", colnames(ortho_gene_mapping))
ortho_gene_mapping[,scaf_col] <- gsub("SCAF", "SUPER", ortho_gene_mapping[,scaf_col]) 

In [20]:
for (i in seq_along(1:nrow(myo_meta))) {

  spec_id <- tolower(gsub("Myotis-", "", myo_meta$Field_Name[i]))
  peak_files <- list.files(path = "resources/peaks", pattern = paste0("^", myo_meta$Abbr[i], ".*_genes.csv"))

  for(j in seq_along(peak_files)) {

    ## Parse gene column:
    spec_peaks <- fread(file.path("resources/peaks", peak_files[j]), data.table = FALSE)
    spec_peaks <- spec_peaks[spec_peaks$Biotype %in% "gene",]
    spec_peaks$Gene <- sapply(strsplit(spec_peaks$Gene, ";", fixed = TRUE), "[", 1)
    spec_peaks$Gene <- gsub("ID=", paste0(spec_id, "_"), spec_peaks$Gene)

    ## Intersect best hit genes with orthologous genes:
    spec_ortho_peaks <- ortho_gene_mapping[,c(1:3, grep(spec_id, colnames(ortho_gene_mapping))),]
    colnames(spec_ortho_peaks)[4] <- "Gene"
    spec_ortho_peaks <- spec_ortho_peaks[spec_ortho_peaks$Gene %in% spec_peaks$Gene,]
    spec_ortho_peaks <- merge(spec_ortho_peaks, spec_peaks, by = "Gene", sort = FALSE)
    fwrite(spec_ortho_peaks, file = paste0(
      "resources/peaks/", gsub(".csv", "_orthologs_only.csv", peak_files[j])
    ))

  }

}